# What each output column is (and why)

**Sampling grid**

* **AZIMUTH, ELEVATION** – angles (in **radians**) of the incoming direction in the **body frame**, swept on a 5° grid (0.087266… rad). You can see exact values like π, π/2, etc. in the table (e.g., -3.14159, -1.5708, …).&#x20;

**Areas / multipliers**

* **EXPOSED** – total exterior area “exposed” for that attitude (m²). It changes step-wise with deployment geometry (values like **0.6003**, **0.6256**, **0.6785** repeat in blocks), which is what you’d expect if different facets/petals become visible/self-shadowed as the direction moves.&#x20;
* **FRONTAL** – projected area **normal to the incoming direction** (m²). It tracks with how “face-on” the spacecraft looks from that az/el, rising from \~0.28 up to \~0.56+ m² as you tip towards broadside.&#x20;
* **DRAG\_MUL** – an **effective drag area multiplier** (unitless) used by the aero model. It scales with FRONTAL but is not identical; values exceed 1.0 (e.g., 1.60) when the projected area and panel orientation maximize free-molecular drag effectiveness. Think of it as the LUT the dynamics code multiplies by ½ρV² and a reference CdA.&#x20;

**Solar radiation pressure (SRP) force vector**

* **SRP\_X\_MUL, SRP\_Y\_MUL, SRP\_Z\_MUL** – the **signed components** of an SRP “force multiplier” vector in body axes, reflecting both illuminated area and incidence/reflection geometry. You’ll notice sign flips with az/el (e.g., X component changing sign across azimuth), and magnitudes around 0.05–0.6 that track how much sunlit area projects along each axis.&#x20;

**Power / thermal**

* **POW\_MUL** – **effective illuminated solar-cell area** (m²) used by the power budget. You can see it snap to specific values that match the areas in your config; e.g., at one sample the value is exactly **0.0448**, which corresponds to the solar area you assigned to a facet in your input. That’s a strong clue this is “∑(solar-cell area × cos(incidence))” clipped at 0.
* **ABS\_MUL** – **solar absorption** multiplier (m²·α equivalent). It follows FRONTAL (and visibility) because it’s essentially the illuminated **absorbing** area (used by the thermal model for absorbed shortwave energy). Values like 0.21 → 0.46 rise with more sun-facing area.&#x20;
* **EMI\_MUL** – **thermal emission** multiplier (m²·ε equivalent) to deep space. It’s the effective radiating area the thermal model uses for **emission**; it also rises when more area is view-factor-visible to space rather than being self-shadowed.&#x20;

> TL;DR mapping you can annotate at the top of the LUT:
>
> * **AZIMUTH, ELEVATION** (rad): direction of “incoming” vector (Sun/flow) in body frame.
> * **EXPOSED** (m²): total visible surface area.
> * **FRONTAL** (m²): projected area normal to incoming direction.
> * **DRAG\_MUL** (–): effective drag multiplier (used with ½ρV²).
> * **SRP\_\*\_MUL** (–): SRP force multipliers along body X/Y/Z.
> * **POW\_MUL** (m²): illuminated solar-cell area for power.
> * **ABS\_MUL** (m²·α): absorbed solar area for thermal.
> * **EMI\_MUL** (m²·ε): emitting area to space for thermal.

# How the Java app uses the files

1. You create an input from the **template** `SATELLITE_GEOMETRY_GENERATOR.txt`, filling in the body size and panel layout. The fields are explained inline (DIMENSIONS, SOLAR\_AREA\_FOR\_FACETS, SOLAR\_PANEL\_n with type/attachment/quadrant/angle/areas).&#x20;
2. Your concrete input `geometry_triple_deployed_input.txt` defines a 0.23×0.23×0.34 m body, 2 petal panels each side (+X bottom, −X top), with inside/outside solar areas of **0.0448** & **0.0224 m²** as applicable, and **TRIPLE** deployables (three petals).&#x20;
3. The app sweeps a grid of az/el directions and writes the **lookup table** `geometry_triple_deployed_output.txt` with the headers above and the numerical values you’re seeing. (The grid steps match 5° increments in radians.)&#x20;

# Why the numbers make sense (quick sanity checks)

* **Angles in radians**: you see exact constants like −π, −π/2, and increments of 0.087266… (5°). ✔️&#x20;
* **POW\_MUL hitting 0.0448**: that matches your **facet solar area** (per the input), confirming it’s an illuminated-area metric used by the power budget. ✔️
* **FRONTAL vs DRAG\_MUL** trend: as the vehicle faces the direction, FRONTAL rises and DRAG\_MUL rises past 1.0 where panel angles maximize effective drag—consistent with a LUT for aero forces. ✔️&#x20;

---

If you want to replace the Java input with YAML and generate the **same** output in Python, we can:

1. Define a YAML schema mirroring the template (body, facet solar areas, list of panels with type/attachment/quadrant/angle/areas/multiplicity).
2. Port the geometry & visibility math to Python (compute which facets/petals are lit/exposed, projected areas, SRP directions, etc.) and write the CSV with the same 11 columns.